In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [10]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.createOrReplaceTempView('home_sales')

# Show DataFrame
spark.sql("select * from home_sales limit 10").show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [16]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """
(SELECT AVG(price) FROM home_sales 
WHERE bedrooms==4)
"""
spark.sql(query).show()
#average price is $299,661


+-----------------+
|       avg(price)|
+-----------------+
|299661.0056283447|
+-----------------+



In [21]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query = """
(SELECT date_built, ROUND(AVG(price), 2) FROM home_sales 
WHERE bedrooms==4 AND bathrooms==3
GROUP BY date_built
ORDER BY date_built
)
"""
spark.sql(query).show()


+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2010|           287529.97|
|      2011|           294656.14|
|      2012|           289533.13|
|      2013|           291870.89|
|      2014|           289935.92|
|      2015|            299517.5|
|      2016|           289090.11|
|      2017|           289100.17|
+----------+--------------------+



In [22]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query = """
(SELECT date_built, ROUND(AVG(price), 2) FROM home_sales 
WHERE bedrooms==3 AND bathrooms==3 AND floors==2 AND sqft_living>=2000
GROUP BY date_built
ORDER BY date_built
)
"""
spark.sql(query).show()


+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2010|           285010.22|
|      2011|           276553.81|
|      2012|           307539.97|
|      2013|           303676.79|
|      2014|           298264.72|
|      2015|           297609.97|
|      2016|            293965.1|
|      2017|           280317.58|
+----------+--------------------+



In [36]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = """
(SELECT view, ROUND(AVG(price), 2) AS avg_price
  FROM home_sales
  GROUP BY view
  HAVING ROUND(AVG(price), 2) >= 350000)
"""
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  51| 788128.21|
|  54| 798684.82|
|  69| 750537.94|
|  87| 1072285.2|
|  73| 752861.18|
|  64| 767036.67|
|  59|  791453.0|
|  85|1056336.74|
|  52| 733780.26|
|  71|  775651.1|
|  98|1053739.33|
|  99|1061201.42|
|  96|1017815.92|
| 100| 1026669.5|
|  70| 695865.58|
|  61| 746877.59|
|  75|1114042.94|
|  78|1080649.37|
|  89|1107839.15|
|  77|1076205.56|
+----+----------+
only showing top 20 rows

--- 0.8197827339172363 seconds ---


In [37]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")


DataFrame[]

In [38]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [39]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query = """
(SELECT view, ROUND(AVG(price), 2) AS avg_price
  FROM home_sales
  GROUP BY view
  HAVING ROUND(AVG(price), 2) >= 350000)
"""
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
|  51| 788128.21|
|  54| 798684.82|
|  69| 750537.94|
|  87| 1072285.2|
|  73| 752861.18|
|  64| 767036.67|
|  59|  791453.0|
|  85|1056336.74|
|  52| 733780.26|
|  71|  775651.1|
|  98|1053739.33|
|  99|1061201.42|
|  96|1017815.92|
| 100| 1026669.5|
|  70| 695865.58|
|  61| 746877.59|
|  75|1114042.94|
|  78|1080649.37|
|  89|1107839.15|
|  77|1076205.56|
+----+----------+
only showing top 20 rows

--- 0.6624279022216797 seconds ---


In [40]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 

df.write.parquet('p_home_sales', mode='overwrite')


In [41]:
# 11. Read the parquet formatted data.
p_df=spark.read.parquet('p_home_sales')


In [42]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_home_sales')

In [43]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

query = """
(SELECT view, ROUND(AVG(price), 2) AS avg_price
  FROM p_home_sales
  GROUP BY view
  HAVING ROUND(AVG(price), 2) >= 350000)
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  51| 788128.21|
|  54| 798684.82|
|  69| 750537.94|
|  87| 1072285.2|
|  73| 752861.18|
|  64| 767036.67|
|  59|  791453.0|
|  85|1056336.74|
|  52| 733780.26|
|  71|  775651.1|
|  98|1053739.33|
|  99|1061201.42|
|  96|1017815.92|
| 100| 1026669.5|
|  70| 695865.58|
|  61| 746877.59|
|  75|1114042.94|
|  78|1080649.37|
|  89|1107839.15|
|  77|1076205.56|
+----+----------+
only showing top 20 rows

--- 0.8532216548919678 seconds ---


In [44]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]

In [45]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False